In [26]:
    import fiona
    import os,os.path
    from shapely.geometry import shape,mapping
    from shapely.ops import transform
    from functools import partial
    import pyproj
    from fiona.crs import from_epsg
    import osmnx as ox
    import matplotlib.pyplot as plt
    import geopandas as gpd
    import glob
    import json
    import numpy as np
    import requests as rq
    import collections
    import logging
    import time
    import dateutil.parser
    %matplotlib inline

In [27]:
# Create a custom logger
logger = logging.getLogger(__name__)

# Create handlers
f_handler = logging.FileHandler('Mapillary.log')
f_handler.setLevel(logging.DEBUG)

# Create formatters and add it to handlers
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(f_handler)

In [28]:
mapillary_Image_ID = 'https://a.mapillary.com/v3/images?client_id=bkJxVGFaQkdEcVY0RnVseWduREE0aTowZWMzMmY1Y2M3MjQ4ZTgx&closeto='
distanceFrag = '&radius=200'
mapillary_Image_url = 'https://images.mapillary.com/' #<image_key>/thumb-640.jpg

In [29]:
city = 'Tokyo'
root = '../Data/'

In [30]:
# data = json.load(open(files[1] , 'rb'))
data = gpd.read_file(root+city+'/shapeFiles/samples.shp')

In [31]:
data

,id,geometry
0,1,POINT (139.44241 35.62744)
1,1,POINT (139.44232 35.62612)
2,1,POINT (139.41958 35.61013)
3,1,POINT (139.42133 35.74586)
4,1,POINT (139.42126 35.74699)
...,...,...
317621,1,POINT (139.63987 35.77190)
317622,1,POINT (139.47959 35.54043)
317623,1,POINT (139.48310 35.53865)
317624,1,POINT (139.57770 35.76024)


In [ ]:
from tqdm import tqdm
count = 1
for point in tqdm(data['geometry']):
    imagesMeta = []
    name= city + str(count) + '.json'
    loc= str(point.x)+','+str(point.y)
#     print(loc)
    finalUrl = mapillary_Image_ID + loc + distanceFrag
    try:
        response = rq.get(finalUrl,timeout=10)
        dictionary = json.loads(response.text)
        imagesMeta = dictionary["features"]
        logger.debug('Working with %d image at location %s'%(count,loc))
    except:
        logger.error('Request Timed out')
#     print(imagesMeta)
    logger.debug('Found %d Images in this file'%(count))
#     print(imagesMeta)
    json.dump(imagesMeta,open(root+ city+ '/' + 'pointsJson/' + str(count) +'.json','w'))
    count+=1

  0%|          | 1092/317626 [58:50<266:22:43,  3.03s/it]

In [ ]:
# for f in files:
#     name = 'Mapillary_'+'_'.join(f.split('/')[-1].split('_')[-2:])
#     logger.debug("Working with %s"%(f))
#     first = json.load(open(f , 'rb'))
#     points = []
#     for k in first:
#         comps = k.split('_')
#         point = (comps[1].strip(),comps[0].strip())
#         points.append(point)
#     imagesMeta = []
#     for point in points:
#         loc= point[0]+','+point[1]
#         finalUrl = mapillary_Image_ID + loc + distanceFrag
#         try:
#             response = rq.get(finalUrl,timeout=5)
#             if response.status_code == 200:
#                 dictionary = json.loads(response.content)
#                 imagesMeta += dictionary["features"]
#             else:
#                 logger.error('Missed point at (%s,%s)'%(point[0],point[1]))
#                 time.sleep(10)
#         except:
#             logger.error('Request Timed out')
#     logger.debug('Found %d Images in this file'%(len(imagesMeta)))
#     logger.debug('Dumping in file: %s'%(name))
#     json.dump(imagesMeta,open(city+name,'wb'))